In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer
import tensorflow as tf
from nltk import word_tokenize

from utils import *
from transforms import *
from diversity_metrics import *

tf.compat.v1.flags.DEFINE_string('f','','')

C:\Users\fabri\Anaconda3\envs\python38\lib\site-packages\spacy\util.py:707: UserWarning: [W095] Model 'en_core_web_sm' (2.3.1) requires spaCy >=2.3.0,<2.4.0 and is incompatible with the current version (3.0.1). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
dataset = load_dataset('glue', 'sst2', split='train')
dataset.rename_column_('sentence', 'text')

Reusing dataset glue (C:\Users\fabri\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


In [30]:
pkl_save(dataset['text'], 'datasets/sst2/INV/text.npy')
pkl_save(dataset['label'], 'datasets/sst2/INV/label.npy')

In [31]:
dataset_paths = parse_path_list('datasets/sst2', 'datasets')

In [34]:
dataset_paths = [x for x in dataset_paths if 'text' in x]

In [37]:
ORIG_text_path = [x for x in dataset_paths if 'ORIG' in x][0]
ORIG_text = npy_load(ORIG_text_path)

In [29]:
for p in dataset_paths:
    DATA_text = npy_load(p)

['datasets/sst2\\ORIG\\label.npy', 'datasets/sst2\\ORIG\\text.npy']

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

In [ ]:
bertscore_metric._compute()

In [43]:
# Init tokenizer and metrics
bertscore_metric = load_metric("bertscore")
bleurt_metric = load_metric("bleurt")

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').



INFO:tensorflow:Reading checkpoint C:\Users\fabri\.cache\huggingface\metrics\bleurt\default\downloads\extracted\9b5629096f074a9f24ea8a214dbc6d5b0a0143ab2a05aa47bf267a266398a79f\bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Performs basic checks...
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Loading model...
INFO:tensorflow:BLEURT initialized.


In [47]:
bleu_score._compute(['I am happy'], ['I am happy'])

{'bleu': 0.0,
 'precisions': [0.7, 0.0, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 10.0,
 'translation_length': 10,
 'reference_length': 1}

In [44]:
# Init tokenizer and metrics
bertscore_metric = load_metric("bertscore")
bleurt_metric = load_metric("bleurt")
bleu_score = load_metric("bleu")

# diversity_metric = TextDiversity(verbose=True)

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint C:\Users\fabri\.cache\huggingface\metrics\bleurt\default\downloads\extracted\9b5629096f074a9f24ea8a214dbc6d5b0a0143ab2a05aa47bf267a266398a79f\bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Performs basic checks...
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Loading model...
INFO:tensorflow:BLEURT initialized.


# Naturalness and Diversity for each Transformation

In [5]:
data = {}

datasets = [('glue', 'sst2'), 'ag_news']

for d in datasets:
      
    # load datasets from huggingface    
    if type(d) == tuple and d[0] == 'glue':
        task = 'sentiment'
        ORIG = load_dataset(d[0], d[1])['train']
        ORIG.rename_column_('sentence', 'text')
        d = d[1]
    else:
        task = 'topic'
        ORIG = load_dataset(d)['train']
            
    # sample n of them to be used for each of the transformations
    n = 100
    idx = np.random.randint(0, len(ORIG['text']), n)
    ORIG_text, ORIG_label = list(np.array(ORIG['text'])[idx]), list(np.array(ORIG['label'])[idx])
    ORIG_text = [str(x) for x in ORIG_text]
    
    data[(d, 'ORIG')] = {'text': ORIG_text, 'label': ORIG_label }
           
    df = init_transforms(task=task)
    
    for t in df.itertuples():
        task_name = t[1]
        tran_type = t[2]
        label_type = t[3]
        transformation = t[4]
        tran_fn = t[5]
        print(task_name, tran_type, label_type, transformation, tran_fn)

        if 'AbstractBatchTransformation' in tran_fn.__class__.__bases__[0].__name__:
            TRAN_text, _ = tran_fn((ORIG_text, ORIG_label))
            TRAN_text = TRAN_text[0].tolist()
        else:
            TRAN_text = []
            for X in ORIG_text:
                new_X = tran_fn(X)[0]
                if type(new_X) == list:
                    new_X = new_X[0]
                TRAN_text.append(new_X)
        TRAN_text = [str(x) for x in TRAN_text]
        data[(d, transformation)] = {'text': TRAN_text}
        

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)


sentiment INV hard ExpandContractions <transformations.text.contraction.expand_contractions.ExpandContractions object at 0x000002A3EE09DD30>
sentiment INV hard ContractContractions <transformations.text.contraction.contract_contractions.ContractContractions object at 0x000002A3EE09DCD0>
sentiment INV hard Emojify <transformations.text.emoji.emojify.Emojify object at 0x000002A3EE084640>
sentiment SIB soft AddPositiveEmoji <transformations.text.emoji.emojify.AddPositiveEmoji object at 0x000002A3F59EEA60>
sentiment SIB soft AddNegativeEmoji <transformations.text.emoji.emojify.AddNegativeEmoji object at 0x000002A3EE18E0D0>
sentiment INV hard AddNeutralEmoji <transformations.text.emoji.emojify.AddNeutralEmoji object at 0x000002A3ED45C1C0>
sentiment INV hard Demojify <transformations.text.emoji.demojify.Demojify object at 0x000002A3EE21E1F0>
sentiment SIB soft RemovePositiveEmoji <transformations.text.emoji.demojify.RemovePositiveEmoji object at 0x000002A3EE21E520>
sentiment SIB soft RemoveN

Using custom data configuration default
Reusing dataset ag_news (C:\Users\Fabrice\.cache\huggingface\datasets\ag_news\default\0.0.0\fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)


topic INV hard ExpandContractions <transformations.text.contraction.expand_contractions.ExpandContractions object at 0x000002A3EE181610>
topic INV hard ContractContractions <transformations.text.contraction.contract_contractions.ContractContractions object at 0x000002A3EE181CD0>
topic INV hard Emojify <transformations.text.emoji.emojify.Emojify object at 0x000002A47F065970>
topic INV hard AddPositiveEmoji <transformations.text.emoji.emojify.AddPositiveEmoji object at 0x000002A47F0E2490>
topic INV hard AddNegativeEmoji <transformations.text.emoji.emojify.AddNegativeEmoji object at 0x000002A47F1059D0>
topic INV hard AddNeutralEmoji <transformations.text.emoji.emojify.AddNeutralEmoji object at 0x000002A47F1E2160>
topic INV hard Demojify <transformations.text.emoji.demojify.Demojify object at 0x000002A47F96E220>
topic INV hard RemovePositiveEmoji <transformations.text.emoji.demojify.RemovePositiveEmoji object at 0x000002A47F96E970>
topic INV hard RemoveNegativeEmoji <transformations.text.e

In [7]:
for key, value in data.items():

    print(key)
    
    ORIG_text = data[(key[0], 'ORIG')]['text']
    
    # Naturalness | BERTScore
    print('calculating BERTScores...')

    scores = bertscore_metric._compute(value['text'], ORIG_text, lang="en")
    f1 = scores['f1'].mean().item()
    print('TRAN vs. ORIG BERTScore', f1)
    data[key]['BERTScore'] = f1

    # Naturalness | BLEURT Scores
    print('calculating BLEURT Scores...')

    scores = bleurt_metric._compute(value['text'], ORIG_text)
    score = np.array(scores['scores']).mean()
    print('TRAN vs. ORIG BLEURT Score', score)
    data[key]['BLEURT Score'] = score

    # Diversity | TTR (lexical diversity) 

    # by input_ids (tokenizing replaces many words with [UNK] and reduces lexical diversity)
    value_tok = tokenizer(value['text'], padding=True, truncation=True)
    TTR_tok = get_ttr_by_ids(value_tok['input_ids'])
    print('TRAN TTR_tok: {0:0.2f}%'.format(TTR_tok * 100))
    data[key]['TTR_tok'] = TTR_tok

    # by text
    TTR_text = get_ttr_by_text(value['text'])
    print('TRAN TTR_text: {0:0.2f}%'.format(TTR_text * 100))
    data[key]['TTR_text'] = TTR_text

    # Diversity | TextDiversity (semantic diversity)
    TRAN_out = diversity_metric(value['text'])
    print('TRAN Diversity: {0} ({1:0.4f}%)'.format(TRAN_out['diversity'], TRAN_out['diversity_normalized'] * 100))
    data[key]['diversity'] = TRAN_out['diversity']
    data[key]['diversity_normalized'] = TRAN_out['diversity_normalized']

('sst2', 'ORIG')
calculating BERTScores...
TRAN vs. ORIG BERTScore 1.0
calculating BLEURT Scores...


 50%|██████████████████████████████████████████                                          | 3/6 [00:00<00:00, 25.41it/s]

TRAN vs. ORIG BLEURT Score 1.013520868420601
TRAN TTR_tok: 55.98%
TRAN TTR_text: 56.65%
getting token embeddings...


7it [00:00, 20.82it/s]                                                                                                 


Using n_components=102


...

Using n_components=768


  0%|                                                                                        | 0/10437 [00:00<?, ?it/s]

calculating similarity matrix...


100%|██████████████████████████████████████████████████████████████████████████| 10437/10437 [1:18:36<00:00,  2.21it/s]


TRAN Diversity: 56.79759874888445 (0.5442%)


In [10]:
cols = ['BERTScore', 'BLEURT Score', 'TTR_tok', 'TTR_text', 'diversity', 'diversity_normalized']
df = pd.DataFrame.from_dict(data).T[cols]
df.reset_index(inplace=True)
df.rename(columns={'level_0': 'dataset', 'level_1': 'transform'}, inplace=True)
df

,dataset,transform,BERTScore,BLEURT Score,TTR_tok,TTR_text,diversity,diversity_normalized
0,sst2,ORIG,1,1.01352,0.559804,0.566549,93.5845,0.0917495
1,sst2,ExpandContractions,1,1.01352,0.559804,0.566549,93.6562,0.0918198
2,sst2,ContractContractions,0.99971,1.0095,0.55653,0.566549,92.9728,0.0906168
3,sst2,Emojify,0.948935,0.297655,0.506404,0.59364,99.4813,0.0980111
4,sst2,AddPositiveEmoji,0.921303,0.456128,0.510714,0.603793,77.9021,0.0695555
...,...,...,...,...,...,...,...,...
69,ag_news,HomoglyphSwap,0.751162,-1.55383,0.17508,0.624914,83.0331,0.0114829
70,ag_news,RandomSwap,0.978248,0.51304,0.371163,0.437095,82.0597,0.0162527
71,ag_news,TextMix,0.924825,-0.33368,0.179519,0.220412,84.8616,0.00812929
72,ag_news,SentMix,0.921027,-0.714771,0.179571,0.220186,86.9253,0.00832937


In [11]:
df.to_clipboard(excel=True)

In [16]:
new_y = [0]
y = np.array([[0, 1]])

In [20]:
y[:, new_y] += 1

In [23]:
y = y / y.sum()

# Dev Area

## Naturalness Scores

### BERTScore

In [330]:
bertscore_metric = load_metric("bertscore")

n = 10000
ORIG = ORIG_train_dataset['text'][:n]
INV = INV_train_dataset['text'][:n]
SIB = SIB_train_dataset['text'][:n]

In [331]:
scores = bertscore_metric._compute(ORIG, ORIG, lang="en")
for key, value in list(scores.items())[:-1]:
    print(key, value.mean())

precision tensor(1.)
recall tensor(1.)
f1 tensor(1.)


In [332]:
scores = bertscore_metric._compute(INV, ORIG, lang="en")
for key, value in list(scores.items())[:-1]:
    print(key, value.mean())

precision tensor(0.7802)
recall tensor(0.8215)
f1 tensor(0.7993)


In [333]:
scores = bertscore_metric._compute(SIB, ORIG, lang="en")
for key, value in list(scores.items())[:-1]:
    print(key, value.mean())

precision tensor(0.7920)
recall tensor(0.8252)
f1 tensor(0.8078)


### BLEURT

In [289]:
bleurt_metric = load_metric("bleurt")

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


INFO:tensorflow:Reading checkpoint C:\Users\Fabrice\.cache\huggingface\metrics\bleurt\default\downloads\extracted\0989b2f25cefa5363b32fb8de03c83fe189c82f971eabcd6248d372510de0c71\bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Performs basic checks...
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Loading model...
INFO:tensorflow:BLEURT initialized.


In [290]:
scores = bleurt_metric._compute(ORIG, ORIG)
print('score', np.array(scores['scores']).mean())

score 1.0065633410215378


In [291]:
scores = bleurt_metric._compute(INV, ORIG)
print('score', np.array(scores['scores']).mean())

score -1.563182808160782


In [292]:
scores = bleurt_metric._compute(SIB, ORIG)
print('score', np.array(scores['scores']).mean())

score -1.6391020435094834


## Diversity

### Lexical Diversity via Type-Token Ratio

In [324]:
from nltk import word_tokenize

def get_ttr_by_ids(input_ids):
    '''
    Type Token Ratio (TTR)
    higher -> more diversity
    '''
    token_ids, counts = np.unique(input_ids, return_counts=True)
    idx = np.isin(token_ids, [0, 101, 102], assume_unique=True, invert=True)
    token_ids, counts = token_ids[idx], counts[idx]
    ttr = len(token_ids) / counts.sum() * 100
    return ttr

def get_ttr_by_text(text):
    '''
    Type Token Ratio (TTR)
    higher -> more diversity
    '''
    tokens = [word_tokenize(x) for x in text]
    tokens = [item for sublist in tokens for item in sublist]
    toks, tok_counts = np.unique(tokens, return_counts=True)
    ttr = len(toks) / tok_counts.sum() * 100
    return ttr

In [327]:
n = 60614
print('ORIG TTR: {0:0.2f} %'.format(get_ttr_by_ids(ORIG_train_dataset_tok['input_ids'][:n])))
print('INV  TTR: {0:0.2f} %'.format(get_ttr_by_ids(INV_train_dataset_tok['input_ids'][:n])))
print('SIB  TTR: {0:0.2f} %'.format(get_ttr_by_ids(SIB_train_dataset_tok['input_ids'][:n])))

ORIG TTR: 1.68 %
INV  TTR: 2.08 %
SIB  TTR: 0.92 %


In [328]:
n = 60614
print('ORIG TTR: {0:0.2f} %'.format(get_ttr_by_text(ORIG_train_dataset['text'][:n])))
print('INV  TTR: {0:0.2f} %'.format(get_ttr_by_text(INV_train_dataset['text'][:n])))
print('SIB  TTR: {0:0.2f} %'.format(get_ttr_by_text(SIB_train_dataset['text'][:n])))

ORIG TTR: 2.58 %
INV  TTR: 15.63 %
SIB  TTR: 2.57 %


### Semantic Diversity via Similarity-Sensitive Entropy

See [embedding_distance.colab](https://colab.research.google.com/drive/1lck-HH7fVY6Ml_XPnQh3ka9H42adZ7vp?usp=sharing) and [weighted species diverseity google sheet](https://docs.google.com/spreadsheets/d/1Qctv2zCMMH-9jSLzPtmmkY2X8eTky9EFOjFvJspb0tg/edit?usp=sharing).